In [ ]:
%load_ext autoreload
%autoreload 2


import logging


logging.basicConfig(
    format='%(asctime)s %(name)s %(levelname)-8s %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S')

logger = logging.getLogger("fab_duckdb")
logger.setLevel(logging.DEBUG)



In [ ]:
import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(x);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(x)"
loopstatement = "loop (select * from {{x}}) over (select x from (values('abc'), ('def')) t(x))"
duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement).df()

In [ ]:
# Embedded Loop

import duckdb
import fabduckdb

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(x);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(x)"
loopstatement = "create table xyz as (loop (select * from {{x}}) over (select x from (values('abc'), ('def')) t(x)))"
duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement)
df = duckdb.default_connection.execute("select * from xyz")